In [1]:
%load_ext autoreload
%autoreload 2

In [141]:
import pandas as pd
import time
import numpy as np
from ClusteringPredictiveModel import ClusteringPredictiveModel
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

In [117]:
data = pd.read_csv("data/Road_Traffic_Fine_Management_Process_labeled.csv", sep=",")

#data["event_nr"] = data.groupby("Case ID").cumcount()+1
#data["first_milestone_event"] = data.groupby("Case ID")["Complete Timestamp"].transform(lambda x: x.sort_values(ascending=True).head(1))

In [119]:
data.head()

,Unnamed: 0,Case ID,Activity,Resource,Complete Timestamp,Variant,amount,article,dismissal,expense,lastSent,lifecycle:transition,matricola,notificationType,paymentAmount,points,totalPaymentAmount,vehicleClass,label,event_nr
0,0,A1,Create Fine,561.0,2006/07/24 01:00:00.000,Variant 3,35.0,157.0,NIL,NaN,NaN,complete,NaN,NaN,NaN,0.0,0.0,A,negative,1
1,1,A1,Send Fine,NaN,2006/12/05 01:00:00.000,Variant 3,NaN,NaN,NaN,11.0,NaN,complete,NaN,NaN,NaN,NaN,NaN,NaN,negative,2
2,2,A100,Create Fine,561.0,2006/08/02 01:00:00.000,Variant 1,35.0,157.0,NIL,NaN,NaN,complete,NaN,NaN,NaN,0.0,0.0,A,negative,1
3,3,A100,Send Fine,NaN,2006/12/12 01:00:00.000,Variant 1,NaN,NaN,NaN,11.0,NaN,complete,NaN,NaN,NaN,NaN,NaN,NaN,negative,2
4,4,A100,Insert Fine Notification,NaN,2007/01/15 01:00:00.000,Variant 1,NaN,NaN,NaN,NaN,P,complete,NaN,P,NaN,NaN,NaN,NaN,negative,3


In [143]:
case_id_col = "Case ID"
event_col = "Activity"
timestamp_col = "Complete Timestamp"
label_col = 'label'
pos_label = 'positive'
cat_cols = ['Resource', 'Variant', 'article', 'dismissal', 'lastSent', 'matricola', 'notificationType', 'vehicleClass']
numeric_cols = ['amount', 'expense', 'paymentAmount', 'points', 'totalPaymentAmount', label_col]

n_clusters = 4

In [146]:
# divide into train and test data
np.random.seed(22)
sample_cases = np.random.choice(data[case_id_col].unique(), 1000)
train_cases, test_cases = train_test_split(sample_cases, train_size = 4.0/5, random_state = 22 )
train = data[data[case_id_col].isin(train_cases)]
test = data[data[case_id_col].isin(test_cases)]

In [147]:
# fit model (cluster traces and build classifier for each cluster)
model = ClusteringPredictiveModel(case_id_col, event_col, label_col, timestamp_col, cat_cols, numeric_cols, n_clusters=n_clusters, n_estimators=100, random_state=22, fillna=True)
model.fit(train)

In [148]:
# predict for test set (assign each trace to the closest cluster and predict outcome using the corresponding classifier)
preds = model.predict_proba(test)

In [149]:
roc_auc_score(model.actual, preds)

0.97043261749144105